In [10]:
# Description: test4
# Only get all games from Steam API
import aiohttp
import asyncio
import pandas as pd

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(games):
    games_data = [(game["appid"], game["name"]) for game in games["applist"]["apps"] if game['name']]
    df = pd.DataFrame(games_data, columns=["appid", "name"])
    df.to_csv("games.csv", index=False)

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(games)

if __name__ == "__main__":
    await main()


In [9]:
# Combine all the data, both games and game details
import aiohttp
import asyncio
import pandas as pd
import time

async def fetch_game_details(session, appid):
    async with session.get(f"https://store.steampowered.com/api/appdetails?appids={appid}") as resp:
        return await resp.json()

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(session,games):
    games_data = []
    for game in games["applist"]["apps"][:500]:
        game_details = await fetch_game_details(session, game["appid"])
        '''
        while not game_details:
            print("executing, API limit reached")
            time.sleep(60)
            game_details = await fetch_game_details(session, game["appid"])
        '''
        if game_details[str(game["appid"])]["success"]:
            data = game_details[str(game["appid"])]["data"]
            games_data.append((game["appid"], game["name"], data.get("price_overview", {}).get("initial", 0), data.get("platforms", {}).get("windows", False), data.get("platforms", {}).get("mac", False), data.get("platforms", {}).get("linux", False), data.get("genres", [])))
    df = pd.DataFrame(games_data, columns=["appid", "name", "price", "windows", "mac", "linux", "genres"])
    df.to_csv("games.csv", index=False)

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(session, games)

if __name__ == "__main__":
     await main()

executing, API limit reached
executing, API limit reached
executing, API limit reached
executing, API limit reached
executing, API limit reached


In [13]:
# (Working version) Combine all the data, both games and game details
import aiohttp
import asyncio
import pandas as pd
import time

async def fetch_game_details(session, appid):
    async with session.get(f"https://steamspy.com/api.php?request=appdetails&appid={appid}") as resp:
        return await resp.json()

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(session,games):
    games_data = []
    for game in games["applist"]["apps"][:1000]:
        game_details = await fetch_game_details(session, game["appid"])
        if game_details:
            data = game_details
            games_data.append((game["appid"], game["name"], data.get("price", "0"), data.get("developer", ""), data.get("publisher", ""), data.get("average_forever", 0), data.get("median_forever", 0), data.get("ccu", 0), data.get("genres", [])))
    df = pd.DataFrame(games_data, columns=["appid", "name", "price", "developer", "publisher", "average_forever", "median_forever", "ccu", "genres"])
    df.to_csv("games.csv", index=False)
    return df

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(session, games)

if __name__ == "__main__":
     await main()